<a href="https://colab.research.google.com/github/William-Burbano-Lima/Analisis_Datos_Explorador/blob/main/mqtt-granja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

¡Claro! Vamos a expandir el proyecto para incluir sensores de temperatura, humedad del suelo (moisture), conductividad eléctrica (EC), salinidad, pH, y niveles de nutrientes (N, P, K). Este proyecto es un ejemplo de cómo integrar múltiples sensores en un sistema de monitoreo ambiental para una granja inteligente.

### Proyecto de Monitoreo Ambiental con Múltiples Sensores para una Granja Inteligente

#### 1. **Descripción del Proyecto**

El proyecto tiene como objetivo monitorear múltiples parámetros ambientales y del suelo en una granja inteligente. Los datos de temperatura, humedad del suelo, conductividad eléctrica, salinidad, pH, y niveles de nutrientes (N, P, K) serán recolectados por sensores, enviados a un broker MQTT, y visualizados en una interfaz web.

#### 2. **Componentes del Proyecto**

1. **Sensores**:
   - **Temperatura y Humedad**: DHT22 o DHT11
   - **Humedad del Suelo**: Capacitive Soil Moisture Sensor
   - **Conductividad Eléctrica (EC)**: Analog EC Sensor
   - **Salinidad**: Salinity Sensor
   - **pH**: Analog pH Sensor
   - **Nutrientes (N, P, K)**: NPK Soil Sensor
2. **Dispositivos de Edge Computing**: Raspberry Pi
3. **Broker MQTT**: Ejemplo Mosquitto
4. **Servidor Web**: Flask/Django
5. **Base de Datos**: PostgreSQL
6. **Aplicación Web**: Visualización y análisis de datos

#### 3. **Diagrama de Arquitectura**

```plaintext
+------------------+     +--------------------+     +-------------------+     +----------------+
|   Sensores       | --> | Edge Computing     | --> | Broker MQTT       | --> | Aplicación Web |
| (Temp, Humedad,  |     | (Raspberry Pi)     |     | (Ej. Mosquitto)   |     | (Flask/Django) |
|  EC, Salinidad,  |     +--------------------+     +-------------------+     +----------------+
|  pH, NPK)        |           |                                         |           |
+------------------+           |                                         |           |
                                +-------------> Base de Datos <-----------+
                                               (Ej. PostgreSQL)
```

#### 4. **Implementación del Proyecto**

##### 4.1 Configuración de los Sensores y el Dispositivo de Edge Computing

**Hardware Necesario:**
- Sensores mencionados anteriormente
- Raspberry Pi
- Conexiones eléctricas (cables, resistencias)

**Código en Python para la Raspberry Pi:**

```python
import Adafruit_DHT
import paho.mqtt.client as mqtt
import time

# Simulamos las lecturas de los sensores de EC, Salinidad, pH y NPK
def read_ec_sensor():
    return 1.2  # Ejemplo de valor

def read_salinity_sensor():
    return 0.8  # Ejemplo de valor

def read_ph_sensor():
    return 6.5  # Ejemplo de valor

def read_npk_sensor():
    return {'N': 20, 'P': 15, 'K': 25}  # Ejemplo de valores

# Configuración del sensor DHT22
sensor = Adafruit_DHT.DHT22
pin_temp_hum = 4  # Pin GPIO al que está conectado el sensor DHT22
pin_moisture = 17  # Ejemplo de pin para el sensor de humedad del suelo

# Configuración del broker MQTT
broker = "broker.hivemq.com"
port = 1883
topic = "granja/sensores"

# Crear una instancia del cliente MQTT
client = mqtt.Client()
client.connect(broker, port)

while True:
    # Leer datos del sensor DHT22
    humedad, temperatura = Adafruit_DHT.read_retry(sensor, pin_temp_hum)
    
    # Leer datos de los otros sensores
    humedad_suelo = 50  # Simulamos un valor de humedad del suelo
    ec = read_ec_sensor()
    salinidad = read_salinity_sensor()
    ph = read_ph_sensor()
    npk = read_npk_sensor()

    if humedad is not None and temperatura is not None:
        # Crear mensaje con todos los datos
        mensaje = {
            "temperatura": temperatura,
            "humedad": humedad,
            "humedad_suelo": humedad_suelo,
            "ec": ec,
            "salinidad": salinidad,
            "ph": ph,
            "npk": npk
        }
        client.publish(topic, str(mensaje))
        print(f"Mensaje publicado: {mensaje}")
    else:
        print("Error al leer el sensor DHT22")
    
    # Esperar antes de la siguiente lectura
    time.sleep(10)
```

##### 4.2 Configuración del Broker MQTT

Ya descrito anteriormente, se utiliza Mosquitto como el broker MQTT.

##### 4.3 Configuración del Servidor Web y la Base de Datos

Utilizaremos Flask y PostgreSQL.

**Código en Python para el Servidor Web con Flask:**

```python
from flask import Flask, render_template
import psycopg2

app = Flask(__name__)

# Configuración de la base de datos
conn = psycopg2.connect(
    dbname="granja_db",
    user="usuario",
    password="contraseña",
    host="localhost",
    port="5432"
)

@app.route('/')
def index():
    cur = conn.cursor()
    cur.execute("SELECT * FROM datos_sensores ORDER BY timestamp DESC LIMIT 10")
    rows = cur.fetchall()
    return render_template('index.html', data=rows)

if __name__ == '__main__':
    app.run(debug=True)
```

**Ejemplo de Plantilla HTML (index.html):**

```html
<!DOCTYPE html>
<html>
<head>
    <title>Monitoreo Ambiental</title>
</head>
<body>
    <h1>Datos de Sensores</h1>
    <table border="1">
        <tr>
            <th>Timestamp</th>
            <th>Temperatura (C)</th>
            <th>Humedad (%)</th>
            <th>Humedad del Suelo (%)</th>
            <th>EC (mS/cm)</th>
            <th>Salinidad (g/L)</th>
            <th>pH</th>
            <th>N (mg/kg)</th>
            <th>P (mg/kg)</th>
            <th>K (mg/kg)</th>
        </tr>
        {% for row in data %}
        <tr>
            <td>{{ row[0] }}</td>
            <td>{{ row[1] }}</td>
            <td>{{ row[2] }}</td>
            <td>{{ row[3] }}</td>
            <td>{{ row[4] }}</td>
            <td>{{ row[5] }}</td>
            <td>{{ row[6] }}</td>
            <td>{{ row[7] }}</td>
            <td>{{ row[8] }}</td>
            <td>{{ row[9] }}</td>
        </tr>
        {% endfor %}
    </table>
</body>
</html>
```

##### 4.4 Subscripción a Mensajes MQTT y Almacenamiento en la Base de Datos

**Código para el subscriptor MQTT en el servidor web:**

```python
import paho.mqtt.client as mqtt
import psycopg2
import json

# Configuración del broker MQTT
broker = "broker.hivemq.com"
port = 1883
topic = "granja/sensores"

# Configuración de la base de datos
conn = psycopg2.connect(
    dbname="granja_db",
    user="usuario",
    password="contraseña",
    host="localhost",
    port="5432"
)

def on_message(client, userdata, message):
    data = json.loads(message.payload.decode())
    temperatura = data['temperatura']
    humedad = data['humedad']
    humedad_suelo = data['humedad_suelo']
    ec = data['ec']
    salinidad = data['salinidad']
    ph = data['ph']
    npk = data['npk']

    cur = conn.cursor()
    cur.execute("""
        INSERT INTO datos_sensores (temperatura, humedad, humedad_suelo, ec, salinidad, ph, n, p, k)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (temperatura, humedad, humedad_suelo, ec, salinidad, ph, npk['N'], npk['P'], npk['K']))
    conn.commit()
    print(f"Datos insertados: {data}")

# Crear una instancia del cliente MQTT
client = mqtt.Client()
client.on_message = on_message

# Conectar al broker y suscribirse al topic
client.connect(broker, port)
client.subscribe(topic)
client.loop_forever()
```

### Conclusión

Este proyecto más completo utiliza múltiples sensores para monitorear varios parámetros ambientales y del suelo en una granja inteligente. Los datos se recogen, se envían a través de MQTT, se almacenan en una base de datos y se visualizan en una aplicación web. Esta implementación proporciona una solución robusta para la gestión y optimización de las condiciones agrícolas utilizando tecnología IoT.

Si tienes alguna pregunta adicional o necesitas más detalles sobre alguna parte del proyecto, ¡háznoslo saber!

In [1]:
# prompt: hola mundo en python

print("Hola mundo!")


Hola mundo!
